# DOHMH Restaurant Data Exploration.

**Goal**  
Clean the DOHMH NYC restaurant inspection data, build a restaurant-level table.

**Plan**  
- keep the main restaurant and inspection fields we need
- remove placeholder inspection dates (01/01/1900) and obvious bad rows
- drop restaurants without coordinates (they cannot be used in spatial joins)
- summarize inspection outcomes for each restaurant (grades and scores)
- create inspection-quality labels (for example, high-risk vs low-risk restaurants)
- save a cleaned restaurant table for use with NYPD and PLUTO

In [1]:
import pandas as pd

path = "../data/raw/DOHMH_New_York_City_Restaurant_Inspection_Results_20251209.csv"
dohmh = pd.read_csv(path)

dohmh.shape

(293401, 27)

In [2]:
dohmh.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location
0,50160484,BAOBAO SLIDER,Manhattan,W,59 STREET COLUMBUS CIRCLE,NaN,6466372137,NaN,01/01/1900,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50178742,PARK & BEL,0,70,NaN,NaN,2129732400,NaN,01/01/1900,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,50157995,BROOKLYN DUMPLING SHOP,Queens,110-00,ROCKAWAY BOULEVARD,11420.0,9172327900,NaN,01/01/1900,NaN,...,NaN,40.677665,-73.828758,410.0,32.0,86400.0,4457718.0,4.115430e+09,QN55,POINT (-73.828757893149 40.677664571164)
3,50176674,GOLDEN STEAMER II INC.,Brooklyn,1964,86 STREET,11214.0,6463749320,NaN,01/01/1900,NaN,...,NaN,40.604960,-73.998862,311.0,38.0,28400.0,3166863.0,3.063740e+09,BK28,POINT (-73.998861987543 40.604959714834)
4,50179018,MADRESS LLC,Brooklyn,348,NOSTRAND AVENUE,11216.0,7184128112,NaN,01/01/1900,NaN,...,NaN,40.687825,-73.951073,303.0,36.0,24300.0,3000000.0,3.017970e+09,BK75,POINT (-73.951072925008 40.687825182452)


In [3]:
dohmh.columns.tolist()

['CAMIS',
 'DBA',
 'BORO',
 'BUILDING',
 'STREET',
 'ZIPCODE',
 'PHONE',
 'CUISINE DESCRIPTION',
 'INSPECTION DATE',
 'ACTION',
 'VIOLATION CODE',
 'VIOLATION DESCRIPTION',
 'CRITICAL FLAG',
 'SCORE',
 'GRADE',
 'GRADE DATE',
 'RECORD DATE',
 'INSPECTION TYPE',
 'Latitude',
 'Longitude',
 'Community Board',
 'Council District',
 'Census Tract',
 'BIN',
 'BBL',
 'NTA',
 'Location']

In [4]:
keep_cols = [
    "CAMIS", # Unique restaurant ID in the DOHMH system
    "DBA", # Restaurant name
    "BORO", # Borough where the restaurant is located
    "BUILDING", # Building number of the restaurant address
    "STREET", # Street name of the restaurant addres
    "ZIPCODE", # ZIP code for the restaurant location
    "CUISINE DESCRIPTION", # Type of cuisine served
    "INSPECTION DATE", # Date of the health inspection
    "SCORE", # Numeric inspection score (higher = worse)
    "GRADE", # Letter grade assigned
    "Latitude", # Latitude coordinate of the restaurant
    "Longitude" # Longitude coordinate of the restaurant
]

dohmh_small = dohmh[keep_cols].copy()
dohmh_small.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,CUISINE DESCRIPTION,INSPECTION DATE,SCORE,GRADE,Latitude,Longitude
0,50160484,BAOBAO SLIDER,Manhattan,W,59 STREET COLUMBUS CIRCLE,NaN,NaN,01/01/1900,NaN,NaN,NaN,NaN
1,50178742,PARK & BEL,0,70,NaN,NaN,NaN,01/01/1900,NaN,NaN,NaN,NaN
2,50157995,BROOKLYN DUMPLING SHOP,Queens,110-00,ROCKAWAY BOULEVARD,11420.0,NaN,01/01/1900,NaN,NaN,40.677665,-73.828758
3,50176674,GOLDEN STEAMER II INC.,Brooklyn,1964,86 STREET,11214.0,NaN,01/01/1900,NaN,NaN,40.604960,-73.998862
4,50179018,MADRESS LLC,Brooklyn,348,NOSTRAND AVENUE,11216.0,NaN,01/01/1900,NaN,NaN,40.687825,-73.951073


### Data Cleaning




In [5]:
raw_summary = dohmh["INSPECTION DATE"].describe()
raw_summary

count         293401
unique          1810
top       01/01/1900
freq            3852
Name: INSPECTION DATE, dtype: object

there are 3852 invalid dates in the INSPECTION DATE column set as 01/01/1900, according to the website it means resturants have not yet been inspected

In [6]:
dohmh_small["INSPECTION DATE"] = pd.to_datetime(
    dohmh_small["INSPECTION DATE"],
    errors="coerce"
)

mask_1900 = dohmh_small["INSPECTION DATE"] == pd.Timestamp("1900-01-01")
dohmh_small.loc[mask_1900, "INSPECTION DATE"] = pd.NaT

date_summary = dohmh_small["INSPECTION DATE"].describe()
date_summary 


count                           289549
mean     2024-03-16 14:10:10.939081728
min                2015-09-24 00:00:00
25%                2023-04-17 00:00:00
50%                2024-05-21 00:00:00
75%                2025-03-10 00:00:00
max                2025-12-06 00:00:00
Name: INSPECTION DATE, dtype: object

*Summary of missining and invalid data for dates:*

In [7]:
n_raw_rows = len(dohmh_small)
n_raw_restaurants = dohmh_small["CAMIS"].nunique()

has_real_date = dohmh_small["INSPECTION DATE"].notna()

n_rows_with_real_date = has_real_date.sum()
n_rows_without_real_date = n_raw_rows - n_rows_with_real_date

n_restaurants_with_real_date = dohmh_small.loc[has_real_date, "CAMIS"].nunique()
n_restaurants_without_real_date = n_raw_restaurants - n_restaurants_with_real_date

rows_with_real_pct = n_rows_with_real_date / n_raw_rows * 100
restaurants_with_real_pct = n_restaurants_with_real_date / n_raw_restaurants * 100

summary_dates = {
    "rows_total": n_raw_rows,
    "rows_with_real_date": n_rows_with_real_date,
    "rows_without_real_date": n_rows_without_real_date,
    "rows_with_real_date_pct": round(rows_with_real_pct, 2),
    "restaurants_total": n_raw_restaurants,
    "restaurants_with_real_date": n_restaurants_with_real_date,
    "restaurants_without_real_date": n_restaurants_without_real_date,
    "restaurants_with_real_date_pct": round(restaurants_with_real_pct, 2),
}
summary_dates

{'rows_total': 293401,
 'rows_with_real_date': np.int64(289549),
 'rows_without_real_date': np.int64(3852),
 'rows_with_real_date_pct': np.float64(98.69),
 'restaurants_total': 30605,
 'restaurants_with_real_date': 26753,
 'restaurants_without_real_date': 3852,
 'restaurants_with_real_date_pct': 87.41}

Keeping only rows with real inspection dates:

In [8]:
dohmh_real = dohmh_small[has_real_date].copy()
dohmh_real.shape, dohmh_real["CAMIS"].nunique()

((289549, 12), 26753)

now we need to build a build restaurant-level table

we grouped by CAMIS to collapse the inspection-level data into one row per restaurant. For each restaurant we keep the first observed name and location, the coordinates, the first and last inspection dates, and the number of inspections.


In [9]:
grouped = dohmh_real.groupby("CAMIS", dropna=False)

restaurants = grouped.agg(
    DBA=("DBA", "first"),
    BORO=("BORO", "first"),
    BUILDING=("BUILDING", "first"),
    STREET=("STREET", "first"),
    ZIPCODE=("ZIPCODE", "first"),
    CUISINE_DESCRIPTION=("CUISINE DESCRIPTION", "first"),
    Latitude=("Latitude", "first"),
    Longitude=("Longitude", "first"),
    first_inspection=("INSPECTION DATE", "min"),
    last_inspection=("INSPECTION DATE", "max"),
    num_inspections=("SCORE", "count"),
    avg_score=("SCORE", "mean"),
    worst_score=("SCORE", "max"),
    best_score=("SCORE", "min"),
    last_score=("SCORE", lambda s: s.dropna().iloc[-1] if not s.dropna().empty else None),
    last_grade=("GRADE", lambda s: s.dropna().iloc[-1] if not s.dropna().empty else None),
)

restaurants = restaurants.reset_index()
restaurants.shape, restaurants.head()


((26753, 17),
       CAMIS                          DBA       BORO BUILDING  \
 0  30075445        MORRIS PARK BAKE SHOP      Bronx     1007   
 1  30191841                D.J. REYNOLDS  Manhattan      351   
 2  40356018             RIVIERA CATERERS   Brooklyn     2780   
 3  40356483           WILKEN'S FINE FOOD   Brooklyn     7114   
 4  40356731  TASTE THE TROPICS ICE CREAM   Brooklyn     1839   
 
                STREET  ZIPCODE       CUISINE_DESCRIPTION   Latitude  \
 0  MORRIS PARK AVENUE  10462.0  Bakery Products/Desserts  40.848231   
 1    WEST   57 STREET  10019.0                     Irish  40.767326   
 2    STILLWELL AVENUE  11224.0                  American  40.579896   
 3            AVENUE U  11234.0                Sandwiches  40.620112   
 4     NOSTRAND AVENUE  11226.0           Frozen Desserts  40.640795   
 
    Longitude first_inspection last_inspection  num_inspections  avg_score  \
 0 -73.855972       2023-01-31      2024-11-08               19  19.684211   
 1 -

To join restaurants to NYPD and PLUTO using location, we need latitude and longitude. Restaurants that are missing coordinates cannot be used in the spatial joins.

 so we measure how many restaurants have valid coordinates and what fraction would be dropped if we require them.

In [10]:
restaurants["has_coords"] = restaurants["Latitude"].notna() & restaurants["Longitude"].notna()

n_restaurants_total_rl = len(restaurants)
n_with_coords = restaurants["has_coords"].sum()
n_without_coords = n_restaurants_total_rl - n_with_coords

with_coords_pct = n_with_coords / n_restaurants_total_rl

coord_summary = {
    "restaurants_total_after_dates": n_restaurants_total_rl,
    "restaurants_with_coords": n_with_coords,
    "restaurants_without_coords": n_without_coords,
    "restaurants_with_coords_pct": round(with_coords_pct * 100, 2),
}
coord_summary


{'restaurants_total_after_dates': 26753,
 'restaurants_with_coords': np.int64(26572),
 'restaurants_without_coords': np.int64(181),
 'restaurants_with_coords_pct': np.float64(99.32)}

seeing its only a small fraction without cordinates we can drop them

In [11]:
restaurants_geo = restaurants[restaurants["has_coords"]].copy()
restaurants_geo = restaurants_geo.drop(columns=["has_coords"])

restaurants_geo.shape, restaurants_geo["CAMIS"].nunique()
restaurants_geo.head()


,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,CUISINE_DESCRIPTION,Latitude,Longitude,first_inspection,last_inspection,num_inspections,avg_score,worst_score,best_score,last_score,last_grade
0,30075445,MORRIS PARK BAKE SHOP,Bronx,1007,MORRIS PARK AVENUE,10462.0,Bakery Products/Desserts,40.848231,-73.855972,2023-01-31,2024-11-08,19,19.684211,38.0,10.0,13.0,P
1,30191841,D.J. REYNOLDS,Manhattan,351,WEST 57 STREET,10019.0,Irish,40.767326,-73.984310,2023-04-23,2025-02-20,10,18.400000,24.0,10.0,24.0,A
2,40356018,RIVIERA CATERERS,Brooklyn,2780,STILLWELL AVENUE,11224.0,American,40.579896,-73.982087,2024-04-16,2025-09-17,3,6.666667,10.0,0.0,0.0,A
3,40356483,WILKEN'S FINE FOOD,Brooklyn,7114,AVENUE U,11234.0,Sandwiches,40.620112,-73.906989,2022-01-24,2025-01-06,20,23.650000,35.0,2.0,21.0,A
4,40356731,TASTE THE TROPICS ICE CREAM,Brooklyn,1839,NOSTRAND AVENUE,11226.0,Frozen Desserts,40.640795,-73.948488,2023-01-17,2025-04-30,9,11.333333,12.0,9.0,12.0,A


In [12]:
# Save cleaned DOHMH restaurant data
output_path = '../data/processed/dohmh_restaurants_clean.csv'
restaurants_geo.to_csv(output_path, index=False)

print(f"✓ Saved {len(restaurants_geo):,} restaurant records to: {output_path}")
print(f"✓ Columns: {restaurants_geo.columns.tolist()}")
print("\nDOHMH data cleaning complete!")


✓ Saved 26,572 restaurant records to: ../data/processed/dohmh_restaurants_clean.csv
✓ Columns: ['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'CUISINE_DESCRIPTION', 'Latitude', 'Longitude', 'first_inspection', 'last_inspection', 'num_inspections', 'avg_score', 'worst_score', 'best_score', 'last_score', 'last_grade']

DOHMH data cleaning complete!
